# Home Work Six - ML Zoomcamp 2025

Hi, I'm Norman! This is my submission for Home Work 6 for ML ZoomCamp 2025.

Let’s connect on [LinkedIn](https://www.linkedin.com/in/anormanangel/) and [Twitter](https://x.com/anormanangel) to keep learning together.
